In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
torch.__version__ #compute unified device architecture

'1.12.0+cu116'

**CUDA performs asynchronus operations to support the paralllelism scheme, in the FIFO order the execution of which is invisible to us.**
**Pytorch uses CUDA streams which is a linear sequence of operations for execution on a single device in a deterministic order within a stream.**

<font size=5>**Linear Regression**</font>

**To perform LR using a single neuron we use Indentity function as the activation function, this will result in a single linear neuron**

**Constraint: y = W*X + b; MSE Optimization; Decision Variables: W and b**

In [3]:
#convert numpy form inputs to tensors
def TensorFromNumpy(x):
    x = torch.from_numpy(x)
    return x

In [4]:
def NumpyFromTensor(x):
    x = x.detach().numpy()
    return x

In [7]:
class RegModel():
    def __init__(self,inputdim,outputdim,hiddendim,lr,epochs):
        self.inputdim = inputdim
        self.outputdim = outputdim
        self.hiddendim = hiddendim
        self.learning_rate = lr
        self.epochs = epochs
        self.w1 = torch.rand(self.inputdim,self.hiddendim,requires_grad=True)
        self.b1 = torch.rand(self.hiddendim,self.outputdim,requires_grad=True)
    
    def Train(self,x_train,y_train):
        for iter in range(self.epochs):
            y_pred = x_train.mm(self.w1).clamp(min=0).add(self.b1) # clamping predicted value simulates the ReLU activation
            loss = (y_pred-y_train).pow(2).sum()
            
            if (iter+1) % 100 == 0:
                print(iter+1,loss.item())
            
            loss.backward()
            
            with torch.no_grad():
                self.w1 -= self.w1.grad * self.learning_rate
                self.b1 -= self.b1.grad * self.learning_rate
                self.w1.grad.zero_()
                self.b1.grad.zero_()
                
    def Predict(self,x):
        if type(x) is np.ndarray:
            x = TensorFromNumpy(x)
        pred = x.mm(self.w1).clamp(min=0).add(self.b1)
        return pred
    

<font size=4>**Ridge Regression**</font>

**Uses regularisation to prevent overfitting on training data by adding a penalty function - L2 norm of the reg. coefficients**

In [9]:
class RidgeRegModel():
    def __init__(self,inputdim,outputdim,hiddendim,lr,epochs):
        self.inputdim = inputdim
        self.outputdim = outputdim
        self.hiddendim = hiddendim
        self.learning_rate = lr
        self.epochs = epochs
        self.w1 = torch.rand(self.inputdim,self.hiddendim,requires_grad=True)
        self.b1 = torch.rand(self.hiddendim,self.outputdim,requires_grad=True)
        self.alpha = 0.6 #strength of the regularization
    
    def Train(self,x_train,y_train):
        for iter in range(self.epochs):
            y_pred = x_train.mm(self.w1).add(self.b1)
            penalty = (w1*w1)
            loss = ((y_pred-y_train).pow(2).sum()) + (alpha * penalty)
            
            if (iter+1) % 100 == 0:
                print(iter+1,loss.item())
            
            loss.backward()
            
            with torch.no_grad():
                self.w1 -= self.w1.grad * self.learning_rate
                self.b1 -= self.b1.grad * self.learning_rate
                self.w1.grad.zero_()
                self.b1.grad.zero_()
                
    def Predict(self,x):
        if type(x) is np.ndarray:
            x = TensorFromNumpy(x)
        pred = x.mm(self.w1).add(self.b1)
        return pred